In [ ]:
import pathlib

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.spatial import cKDTree
from scipy.stats import binned_statistic
from scipy.interpolate import interp1d
from tqdm import tqdm

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from totoro.abundance_helpers import elem_to_label
from totoro.config import (cache_path, fig_path, 
                           elem_names as plot_elem_names)
from totoro.potentials import potentials, galpy_potentials, get_mw_potential
from totoro.objective import TorusImagingObjective

In [ ]:
data_name = 'apogee-rgb-loalpha'
this_cache_path = cache_path / data_name

In [ ]:
fiducials = {
    'mdisk_f': 1.,
    'zsun': 20.8,
    'vzsun': 7.78
}

In [ ]:
tbls = {}
for elem in plot_elem_names:
    path = pathlib.Path(this_cache_path / f"optimize-results-{elem}.csv")
    if not path.exists():
        continue
    tbls[elem] = at.Table.read(path)
    
len(tbls)

In [ ]:
fiducials = {
    'mdisk_f': 1.,
    'zsun': 20.8,
    'vzsun': 7.78
}

colcols = [
    ('mdisk_f', 'zsun'), 
    ('mdisk_f', 'vzsun'), 
    ('zsun', 'vzsun')
]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for elem in tbls:
    for i, (col1, col2) in enumerate(colcols):
        ax = axes[i]
        ax.plot(tbls[elem][col1], tbls[elem][col2],
                ls='none', marker='o', mew=0, ms=4, 
                label=elem_to_label(elem))
axes[0].legend()

axes[0].set_xlabel(r'${\rm M}_{\rm disk} / {\rm M}_{\rm disk}^\star$')
axes[1].set_xlabel(r'${\rm M}_{\rm disk} / {\rm M}_{\rm disk}^\star$')
axes[2].set_xlabel(r'$z_\odot$ [pc]')

axes[0].set_ylabel(r'$z_\odot$ [pc]')
axes[1].set_ylabel(r'$v_{z,\odot}$ ' + f'[{u.km/u.s:latex_inline}]')
axes[2].set_ylabel(r'$v_{z,\odot}$ ' + f'[{u.km/u.s:latex_inline}]')

for ax, (col1, col2) in zip(axes, colcols):
    ax.axvline(fiducials[col1], zorder=-10, color='#aaaaaa', linestyle='--')
    ax.axhline(fiducials[col2], zorder=-10, color='#aaaaaa', linestyle='--')

fig.set_facecolor('w')
    
fig.tight_layout()

In [ ]:
# From https://matplotlib.org/devdocs/gallery/statistics/confidence_ellipse.html
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

tab10 = plt.get_cmap('tab10')


def confidence_ellipse(x, y, ax, n_std=1.0, facecolor='none', **kwargs):
    cov = np.cov(x, y)
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


def plot_cov_ellipse(m, C, ax, n_std=1.0, facecolor='none', **kwargs):
    pearson = C[0, 1] / np.sqrt(C[0, 0] * C[1, 1])
    
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(n_std * np.sqrt(C[0, 0]), 
               n_std * np.sqrt(C[1, 1])) \
        .translate(m[0], m[1])

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
means = np.zeros((len(plot_elem_names), 3))
covs = np.zeros((len(plot_elem_names), 3, 3))
for j, elem in enumerate(plot_elem_names):
    mask = (np.isfinite(tbls[elem]['mdisk_f']) & 
            np.isfinite(tbls[elem]['zsun']) &
            np.isfinite(tbls[elem]['vzsun']))
    X = np.stack((tbls[elem]['mdisk_f'][mask], 
                  tbls[elem]['zsun'][mask],
                  tbls[elem]['vzsun'][mask]))

    covs[j] = np.cov(X)
    means[j] = np.mean(X, axis=1)
    

# _mask = ~np.isin(np.array(elem_names), ['SI_FE', 'MG_FE'])
_mask = np.array(plot_elem_names) != 'sdf'
C = np.linalg.inv(np.sum([np.linalg.inv(cov) 
                          for cov in covs[_mask]], axis=0))
m = np.sum([C @ np.linalg.inv(cov) @ mean 
            for mean, cov in zip(means[_mask], covs[_mask])], 
           axis=0)
print(m)
print(C)

In [ ]:
logdets = [np.linalg.slogdet(cov)[1] for cov in covs]
norm = mpl.colors.Normalize(vmin=np.min(logdets), 
                            vmax=np.max(logdets), 
                            clip=True)
norm2 = mpl.colors.Normalize(vmin=-0.2, vmax=1.1)
def get_alpha(ld):
    return norm2(1 - norm(ld))

In [ ]:
for i, name in enumerate(fiducials.keys()):
    print(f"{m[i]:.2f} +/- {np.sqrt(C[i,i]):.2f}")

In [ ]:
m[0]

In [ ]:
best_potential = get_mw_potential(m[0] * potentials['1.0']['disk'].parameters['m'])
print(f"{best_potential['disk'].parameters['m']:.2e}")
menc1 = best_potential['disk'].mass_enclosed([1.,0,0] * (8.1*u.kpc))[0]
print(f"disk M(<8.1): {menc1:.2e}")

print(f"nfw scale mass: {best_potential['halo'].parameters['m'][0]:.2e}")

rs = best_potential['halo'].parameters['r_s']
menc2 = best_potential['halo'].mass_enclosed([1.,0,0] * (8.1*u.kpc))[0]
print(f"nfw M(<8.1): {menc2:.2e}")

print(f"mdisk/mhalo(<8.1): {menc1 / menc2:.1e}")

In [ ]:
# percent:
for i, name in enumerate(fiducials.keys()):
    print(np.sqrt(C[i,i]) / m[i])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for elem, color, logdet in zip(elem_names, tab10.colors, logdets):
    col1 = 'mdisk_f'
    col2 = 'vzsun'
        
    mask = np.isfinite(tbls[elem][col1]) & np.isfinite(tbls[elem][col2])
    ell = confidence_ellipse(tbls[elem][col1][mask], 
                             tbls[elem][col2][mask], 
                             ax,
                             n_std=1.,
                             linewidth=0, facecolor=color, 
                             alpha=get_alpha(logdet), 
                             label=elem_to_label(elem))

    ell = confidence_ellipse(tbls[elem][col1][mask], 
                             tbls[elem][col2][mask], 
                             ax,
                             n_std=2.,
                             linewidth=0, facecolor=color, 
                             alpha=get_alpha(logdet) / 2)

mm = np.delete(m, 1)
CC = np.delete(np.delete(C, 1, axis=0), 1, axis=1)
ell = plot_cov_ellipse(mm, CC, ax=ax, 
                       n_std=1.,
                       linewidth=0, facecolor='k', 
                       alpha=0.85, label='joint', zorder=100)

ell = plot_cov_ellipse(mm, CC, ax=ax, 
                       n_std=2.,
                       linewidth=0, facecolor='k', 
                       alpha=0.3, zorder=100)
        
ax.set_xlim(0.4, 1.8)
ax.set_ylim(0, 15)

ax.legend()

ax.set_xlabel(r'${\rm M}_{\rm disk} / {\rm M}_{\rm disk}^\star$')
ax.set_ylabel(r'$v_{z,\odot}$ ' + f'[{u.km/u.s:latex_inline}]')

ax.axvline(fiducials[col1], zorder=-10, color='#aaaaaa', linestyle='--')
ax.axhline(fiducials[col2], zorder=-10, color='#aaaaaa', linestyle='--')

fig.set_facecolor('w')
    
fig.tight_layout()

fig.savefig(fig_path / 'M-vz-error-ellipses.pdf')

In [ ]:
colcols = [
    ('mdisk_f', 'zsun'), 
    ('vzsun', 'zsun')
]

fig, axes = plt.subplots(1, 2, figsize=(9.5, 5),
                         sharey=True)

for elem, color, logdet in zip(elem_names, tab10.colors, logdets):
    for i, (col1, col2) in enumerate(colcols):
        ax = axes[i]
        
        mask = np.isfinite(tbls[elem][col1]) & np.isfinite(tbls[elem][col2])
        ell = confidence_ellipse(tbls[elem][col1][mask], 
                                 tbls[elem][col2][mask], 
                                 ax,
                                 n_std=1.,
                                 linewidth=0, facecolor=color, 
                                 alpha=get_alpha(logdet), 
                                 label=elem_to_label(elem))
        
        ell = confidence_ellipse(tbls[elem][col1][mask], 
                                 tbls[elem][col2][mask], 
                                 ax,
                                 n_std=2.,
                                 linewidth=0, facecolor=color, 
                                 alpha=get_alpha(logdet) / 2)

# M, z
ell = plot_cov_ellipse(m[:2], C[:2, :2], ax=axes[0], 
                       n_std=1.,
                       linewidth=0, facecolor='k', 
                       alpha=0.85, label='joint', zorder=100)

ell = plot_cov_ellipse(m[:2], C[:2, :2], ax=axes[0], 
                       n_std=2.,
                       linewidth=0, facecolor='k', 
                       alpha=0.3, zorder=100)

# vz, z
ell = plot_cov_ellipse(m[[2, 1]], np.flipud(np.fliplr(C[1:, 1:])), ax=axes[1], 
                       n_std=1.,
                       linewidth=0, facecolor='k', 
                       alpha=0.85, label='joint', zorder=100)

ell = plot_cov_ellipse(m[[2, 1]], np.flipud(np.fliplr(C[1:, 1:])), ax=axes[1], 
                       n_std=2.,
                       linewidth=0, facecolor='k', 
                       alpha=0.3, zorder=100)
        
axes[0].set_xlim(0.4, 1.8)
axes[1].set_xlim(0, 15)
axes[0].set_ylim(-40, 40)

axes[1].legend()

axes[0].set_xlabel(r'${\rm M}_{\rm disk} / {\rm M}_{\rm disk}^\star$')
axes[1].set_xlabel(r'$v_{z,\odot}$ ' + f'[{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(r'$z_\odot$ [pc]')

for ax, (col1, col2) in zip(axes, colcols):
    ax.axvline(fiducials[col1], zorder=-10, color='#aaaaaa', linestyle='--')
    ax.axhline(fiducials[col2], zorder=-10, color='#aaaaaa', linestyle='--')

fig.set_facecolor('w')
    
fig.tight_layout()

fig.savefig(fig_path / 'M-z-vz-z-error-ellipses.pdf')

### Old plot:

In [ ]:
colcols = [
    ('mdisk_f', 'zsun'), 
    ('mdisk_f', 'vzsun'), 
    ('zsun', 'vzsun')
]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for elem, color, logdet in zip(elem_names, tab10.colors, logdets):
    for i, (col1, col2) in enumerate(colcols):
        ax = axes[i]
        
        mask = np.isfinite(tbls[elem][col1]) & np.isfinite(tbls[elem][col2])
        ell = confidence_ellipse(tbls[elem][col1][mask], 
                                 tbls[elem][col2][mask], 
                                 ax,
                                 n_std=1.,
                                 linewidth=0, facecolor=color, 
                                 alpha=get_alpha(logdet), 
                                 label=elem_to_label(elem))
        
        ell = confidence_ellipse(tbls[elem][col1][mask], 
                                 tbls[elem][col2][mask], 
                                 ax,
                                 n_std=2.,
                                 linewidth=0, facecolor=color, 
                                 alpha=get_alpha(logdet) / 2)

for j, i in enumerate([2, 1, 0]):
    mm = np.delete(m, i)
    CC = np.delete(np.delete(C, i, axis=0), i, axis=1)
    ell = plot_cov_ellipse(mm, CC, ax=axes[j], 
                           n_std=1.,
                           linewidth=0, facecolor='k', 
                           alpha=0.85, label='joint', zorder=100)
    
    ell = plot_cov_ellipse(mm, CC, ax=axes[j], 
                           n_std=2.,
                           linewidth=0, facecolor='k', 
                           alpha=0.3, zorder=100)
        
axes[0].set_xlim(0.4, 1.8)
axes[1].set_xlim(0.4, 1.8)
axes[2].set_xlim(-80, 30)

axes[0].set_ylim(-40, 40)
axes[1].set_ylim(0, 15)
axes[2].set_ylim(0, 15)
        
axes[0].legend()

axes[0].set_xlabel(r'${\rm M}_{\rm disk} / {\rm M}_{\rm disk}^\star$')
axes[1].set_xlabel(r'${\rm M}_{\rm disk} / {\rm M}_{\rm disk}^\star$')
axes[2].set_xlabel(r'$z_\odot$ [pc]')

axes[0].set_ylabel(r'$z_\odot$ [pc]')
axes[1].set_ylabel(r'$v_{z,\odot}$ ' + f'[{u.km/u.s:latex_inline}]')
axes[2].set_ylabel(r'$v_{z,\odot}$ ' + f'[{u.km/u.s:latex_inline}]')

for ax, (col1, col2) in zip(axes, colcols):
    ax.axvline(fiducials[col1], zorder=-10, color='#aaaaaa', linestyle='--')
    ax.axhline(fiducials[col2], zorder=-10, color='#aaaaaa', linestyle='--')

fig.set_facecolor('w')
    
fig.tight_layout()

# fig.savefig(fig_path / 'param-contours.pdf')